### Gemini 2.0 flash lite VS Tuned Gemini 2.0 flash lite

In [13]:
# --- Cell 1: Imports and Setup ---
import os
import sys
import logging
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown # For displaying DataFrames and Markdown

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger("google.api_core").setLevel(logging.WARNING)
logging.getLogger("google.auth").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING) # Suppress PIL logs if noisy

# Add src directory to Python path to import project modules
# Assumes notebook is in the project root directory (e.g., GDG_HACKATHON/)
project_root = os.path.abspath('.') # Use '.' if notebook is in root
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"Added {src_path} to sys.path")

# Import project modules
try:
    import config
    import utils
    import vllm_handler
    # Import specific function to check signature easily
    from vllm_handler import analyze_content
    import inspect # To inspect function signature

    print("Project modules (config, utils, vllm_handler) imported successfully.")

    # Verify the handler has the modified function signature
    sig = inspect.signature(analyze_content)
    if 'model_id_override' not in sig.parameters:
         print("\nCRITICAL WARNING: vllm_handler.analyze_content is missing the 'model_id_override' parameter!")
         print("Please update src/vllm_handler.py and RESTART THE KERNEL before proceeding.")
         # Optionally raise an error to stop execution
         # raise AttributeError("analyze_content function signature is incorrect.")
    else:
         print("Verified: vllm_handler.analyze_content has 'model_id_override' parameter.")


except ImportError as e:
    print(f"Error importing project modules: {e}")
    print("Please ensure:")
    print("1. This notebook is saved in the project's root directory (GDG_HACKATHON/).")
    print("2. The src directory and its files (config.py, utils.py, vllm_handler.py) exist.")
    # Stop execution if modules can't be loaded
    raise

Project modules (config, utils, vllm_handler) imported successfully.

CRITICAL WARNING: vllm_handler.analyze_content is missing the 'model_id_override' parameter!
Please update src/vllm_handler.py and RESTART THE KERNEL before proceeding.


In [14]:
# --- Cell 2: Configuration ---

BASE_MODEL_ID = "gemini-2.0-flash-lite-001"
# IMPORTANT: Verify this is your correct tuned model ID and region
TUNED_MODEL_ID = "projects/248124319532/locations/europe-west4/models/8219698240602243072"

# Define output file paths (relative to the project root)
output_dir = os.path.join(project_root, 'outputs')
COMPARISON_OUTPUT_FILENAME = "comparison_results.json"
GRAPH_OUTPUT_FILENAME = "comparison_graph.png"
COMPARISON_TABLE_FILENAME = "comparison_table.csv" # For saving the DataFrame

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print(f"Base Model ID: {BASE_MODEL_ID}")
print(f"Tuned Model ID: {TUNED_MODEL_ID}")
print(f"Output Directory: {output_dir}")
print(f"Input Directory: {config.INPUT_DIR}") # From imported config
print(f"Project ID: {config.GCP_PROJECT_ID}")
print(f"Region: {config.GCP_REGION}") # Make sure this matches the model region (europe-west4)

Base Model ID: gemini-2.0-flash-lite-001
Tuned Model ID: projects/248124319532/locations/europe-west4/models/8219698240602243072
Output Directory: /home/harishi/common_drive/Downloads/projects/GDG_HACKATHON/my_learning/outputs
Input Directory: /home/harishi/common_drive/Downloads/projects/GDG_HACKATHON/inputs
Project ID: maximal-cider-458611-r5
Region: europe-west4


In [15]:
# --- Cell 3: Initialize Vertex AI ---

# Initialize Vertex AI SDK (important before making API calls)
# vllm_handler should handle the actual vertexai.init call via initialize_vertex_ai()
# We call it once here to ensure it's done before the loop.
if vllm_handler.initialize_vertex_ai():
    print("Vertex AI SDK initialized successfully for comparison.")
else:
    print("ERROR: Vertex AI SDK initialization failed. Cannot proceed.")
    # Optionally raise an error to stop the notebook
    # raise RuntimeError("Vertex AI SDK failed to initialize.")

Vertex AI SDK initialized successfully for comparison.


In [ ]:
# --- Cell 4: Run Comparison Analysis ---

comparison_results_list = [] # Store results as a list of dicts for easier DataFrame creation
base_model_times = []
tuned_model_times = []
files_processed_base = 0
files_processed_tuned = 0
files_error_base = 0
files_error_tuned = 0

# Get Input Files
# Use absolute path for input directory for robustness
abs_input_dir = os.path.abspath(config.INPUT_DIR)
input_files = utils.get_input_files(abs_input_dir)


if not input_files:
    logging.warning(f"No input files found in {abs_input_dir}. Exiting comparison.")
else:
    logging.info(f"Found {len(input_files)} files to process.")

    # --- Run Base Model ---
    logging.info(f"\n--- Running Base Model: {BASE_MODEL_ID} ---")
    total_start_time_base = time.time()
    for file_path in input_files: # file_path is now absolute
        relative_path = os.path.relpath(file_path, project_root) # Get path relative to project root
        logging.info(f"Processing {relative_path} with BASE model...")
        file_start_time = time.time()

        # Call analyze_content, overriding the model ID
        # Use the imported function directly for clarity
        result_str_base = analyze_content(file_path, model_id_override=BASE_MODEL_ID)
        file_end_time = time.time()
        duration = file_end_time - file_start_time
        files_processed_base += 1

        result_entry = {
            "file": relative_path,
            "base_model_output": result_str_base,
            "base_model_time_sec": None,
            "tuned_model_output": "N/A", # Placeholder
            "tuned_model_time_sec": None, # Placeholder
        }

        if result_str_base.startswith("Error:"):
            logging.error(f"Base model error for {relative_path}: {result_str_base}")
            files_error_base += 1
        else:
            base_model_times.append(duration)
            result_entry["base_model_time_sec"] = duration
            logging.info(f"Base model success for {relative_path} in {duration:.2f}s")

        comparison_results_list.append(result_entry) # Add entry even if error occurred

    total_end_time_base = time.time()
    logging.info(f"Base model run finished. Total time: {total_end_time_base - total_start_time_base:.2f}s")
    logging.info(f"Base model: {files_processed_base - files_error_base} successful, {files_error_base} errors.")

    # --- Run Tuned Model ---
    logging.info(f"\n--- Running Tuned Model: {TUNED_MODEL_ID} ---")
    total_start_time_tuned = time.time()

    # Find the corresponding entry in the list to update
    for i, file_path in enumerate(input_files): # file_path is absolute
        relative_path = os.path.relpath(file_path, project_root) # Get path relative to project root
        logging.info(f"Processing {relative_path} with TUNED model...")
        file_start_time = time.time()

        # Call analyze_content using the tuned model ID (explicitly)
        # Ensure the 'AttributeError' is fixed before this step!
        result_str_tuned = analyze_content(file_path, model_id_override=TUNED_MODEL_ID)
        file_end_time = time.time()
        duration = file_end_time - file_start_time
        files_processed_tuned += 1

        # Update the existing dictionary in the list
        # Find the dict for the current file
        entry_to_update = next((item for item in comparison_results_list if item["file"] == relative_path), None)

        if entry_to_update:
             entry_to_update["tuned_model_output"] = result_str_tuned
             if result_str_tuned.startswith("Error:"):
                 logging.error(f"Tuned model error for {relative_path}: {result_str_tuned}")
                 files_error_tuned += 1
             else:
                 tuned_model_times.append(duration)
                 entry_to_update["tuned_model_time_sec"] = duration
                 logging.info(f"Tuned model success for {relative_path} in {duration:.2f}s")
        else:
             # This case should ideally not happen if base run added all files
             logging.error(f"Could not find result entry for {relative_path} to update tuned model output.")
             # Optionally create a new entry if needed, though it indicates a logic issue
             # comparison_results_list.append({
             #     "file": relative_path,
             #     "base_model_output": "N/A",
             #     "base_model_time_sec": None,
             #     "tuned_model_output": result_str_tuned,
             #     "tuned_model_time_sec": duration if not result_str_tuned.startswith("Error:") else None,
             # })


    total_end_time_tuned = time.time()
    logging.info(f"Tuned model run finished. Total time: {total_end_time_tuned - total_start_time_tuned:.2f}s")
    logging.info(f"Tuned model: {files_processed_tuned - files_error_tuned} successful, {files_error_tuned} errors.")

    logging.info("\nComparison processing finished.")

2025-05-03 05:40:22,826 - INFO - Recursively scanning for supported files in: /home/harishi/common_drive/Downloads/projects/GDG_HACKATHON/inputs
2025-05-03 05:40:22,828 - INFO - Found 52 supported files across all subdirectories.
2025-05-03 05:40:22,829 - INFO - Found 52 files to process.
2025-05-03 05:40:22,829 - INFO - 
--- Running Base Model: gemini-2.0-flash-lite-001 ---
2025-05-03 05:40:22,830 - INFO - Processing ../inputs/jpeg/1.jpeg with BASE model...


NameError: name 'set_model_id' is not defined

In [ ]:
# --- Cell 5: Save Full Results to JSON ---

# Save the detailed comparison results
comparison_output_path = os.path.join(output_dir, COMPARISON_OUTPUT_FILENAME)
if comparison_results_list: # Only save if there are results
    try:
        with open(comparison_output_path, 'w', encoding='utf-8') as f:
            # Use indent for readability
            json.dump(comparison_results_list, f, indent=4, ensure_ascii=False)
        print(f"Full comparison results saved to: {comparison_output_path}")
    except Exception as e:
        print(f"Error saving comparison JSON: {e}")
else:
    print("No comparison results generated to save.")

In [ ]:
# --- Cell 6: Create and Display Pandas DataFrame ---

if comparison_results_list:
    # Convert the list of dictionaries to a Pandas DataFrame
    comparison_df = pd.DataFrame(comparison_results_list)

    # Optionally save the DataFrame to CSV
    comparison_table_path = os.path.join(output_dir, COMPARISON_TABLE_FILENAME)
    try:
        comparison_df.to_csv(comparison_table_path, index=False)
        print(f"Comparison table saved to: {comparison_table_path}")
    except Exception as e:
        print(f"Error saving comparison CSV: {e}")


    # Display the first few rows of the DataFrame in the notebook
    print("\n--- Comparison Results Table (First 5 Rows) ---")
    # Configure pandas display options for better readability if needed
    pd.set_option('display.max_colwidth', 100) # Show more text in columns
    pd.set_option('display.max_rows', 10)      # Limit rows displayed initially
    display(comparison_df.head())

    # Display basic stats for timing columns (ignoring NaNs)
    print("\n--- Timing Statistics (seconds per successful file) ---")
    # Calculate stats only on non-error rows where time is not None
    timing_stats = comparison_df[['base_model_time_sec', 'tuned_model_time_sec']].dropna().describe()
    if not timing_stats.empty:
         display(timing_stats)
    else:
         print("No successful timing data available for statistics.")

else:
    print("No comparison results to display in DataFrame.")

In [ ]:
# --- Cell 7: Generate and Display Comparison Graph ---

# Calculate average times only from successful runs
avg_time_base_ms = None
avg_time_tuned_ms = None

if base_model_times:
    avg_time_base_ms = (sum(base_model_times) / len(base_model_times)) * 1000 # Convert to ms
    print(f"Average Time Base Model (Successful Runs): {avg_time_base_ms:.2f} ms ({len(base_model_times)} files)")
else:
    print("No successful runs recorded for the base model.")

if tuned_model_times:
    avg_time_tuned_ms = (sum(tuned_model_times) / len(tuned_model_times)) * 1000 # Convert to ms
    print(f"Average Time Tuned Model (Successful Runs): {avg_time_tuned_ms:.2f} ms ({len(tuned_model_times)} files)")
else:
    print("No successful runs recorded for the tuned model.")


# Plotting only if both models had successful runs
if avg_time_base_ms is not None and avg_time_tuned_ms is not None:
    models = [f'Base\n({BASE_MODEL_ID})', f'Tuned\n(ID: ...{TUNED_MODEL_ID[-6:]})'] # Shorten labels
    avg_times = [avg_time_base_ms, avg_time_tuned_ms]

    fig, ax = plt.subplots(figsize=(8, 6)) # Create figure and axes objects
    bars = ax.bar(models, avg_times, color=['skyblue', 'lightgreen'])
    ax.set_ylabel('Average Time per File (ms)')
    ax.set_title('Model Speed Comparison (Avg. Time for Successful Analyses)')
    # Set y-limit dynamically based on the larger average time
    ax.set_ylim(0, max(avg_times) * 1.2) # Add 20% padding above the taller bar

    # Add text labels to bars
    for bar in bars:
        yval = bar.get_height()
        # Format the label to have 0 decimal places for milliseconds
        ax.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.0f} ms', va='bottom', ha='center', fontsize=9)

    # Save the graph
    graph_path = os.path.join(output_dir, GRAPH_OUTPUT_FILENAME)
    try:
        plt.savefig(graph_path)
        print(f"\nComparison graph saved to: {graph_path}")
    except Exception as e:
        print(f"\nError saving comparison graph: {e}")

    # Display the plot inline in the notebook
    plt.show()

else:
    print("\nCannot generate time comparison graph: Insufficient successful runs for one or both models.")